In [1]:
import os
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms

# np.random.seed(42)  # for reproducibility
# Get the current working directory
current_directory = os.getcwd()

# Construct the relative path to prediction RUL file
rul_filename = "RUL_consultancy_predictions_A3-2.csv"
rul_path = os.path.join(current_directory, rul_filename)

# Read the CSV file
rul_df = pd.read_csv(rul_path)
print(rul_df.head(5))


  RUL;id
0  135;1
1  125;2
2   63;3
3  100;4
4  103;5


In [2]:
# Split the 'RUL;id' column into two separate columns
rul_df[['RUL', 'id']] = rul_df['RUL;id'].str.split(';', expand=True)

# Convert values to integers
rul_df['RUL'] = rul_df['RUL'].astype(int)
rul_df['id'] = rul_df['id'].astype(int)

# Drop the original 'RUL;id' column
rul_df.drop(columns=['RUL;id'], inplace=True)

# Print the modified DataFrame
print(rul_df)

    RUL   id
0   135    1
1   125    2
2    63    3
3   100    4
4   103    5
..  ...  ...
95  140   96
96  109   97
97   87   98
98  127   99
99   24  100

[100 rows x 2 columns]


In [3]:
# Define constants
M = len(rul_df) # number of engines
G = 4 # Total number of teams (2 type A, 2 type B)
TEAM_A = ["T1", "T3"]
TEAM_B = ["T2", "T4"]
T = 30 # Planning horizon in days
MAX_DAILY_COST = 250

In [4]:
# Define maintenance times for teams A and B
maintenance_duration_a = [4 if i < 20 else 3 if 20 <= i < 55 else 2 if 55 <= i < 80 else 8 for i in range(1, M + 1)]
maintenance_duration_b = [time_a + 1 if i < 25 else time_a + 2 if 25 <= i < 70 else time_a + 1 for i, time_a in enumerate(maintenance_duration_a, start=1)]


# Define engine costs
engine_costs = [4 if i < 21 else 3 if 21 <= i < 31 else 2 if 31 <= i < 46 else 5 if 46 <= i < 81 else 6 for i in range(1, M + 1)]

print(engine_costs)
print(maintenance_duration_a)
print(maintenance_duration_b)


[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]


## Individual

In [8]:

# Penalty cost function
def penalty_cost(engine_cost, days_past_due):
    return min(engine_cost * (days_past_due)**2, MAX_DAILY_COST)


def generate_random_schedule(T):
    individual = []
    # Track the availability of each team (A and B) on each day
    team_availability = {day: {'A': 0, 'B': 0} for day in range(1, T + 1)}
    
    # Apply filter to allocate allocate teams to engines that have a predicted safety due date of less than T = 30.

    filtered_df = rul_df[rul_df['RUL'] <= T].copy()
    engine_ids = list(filtered_df['id'])
    random.shuffle(engine_ids)
    
    print("Engine IDs:", engine_ids)
    
    # Randomly assign teams to engines within the planning horizon
    for engine_id in engine_ids:
        print("\nProcessing Engine ID:", engine_id)
    
        team_type = np.random.choice(['A', 'B'])  # Assume team type is A for this example
        print("Selected Team Type:", team_type)
    
        # Determine maintenance duration based on team type
        maintenance_days = maintenance_duration_a[engine_id - 1] if team_type == 'A' else maintenance_duration_b[engine_id - 1]
        print("Maintenance Days Required:", maintenance_days)
    
        valid_start_dates = []

        # Search for valid start dates with continuous available days for maintenance duration
        for start_date in range(1, T - maintenance_days + 2):
            is_valid = True
            for day in range(start_date, start_date + maintenance_days):
                if team_availability[day][team_type] >= 2:  # Check if max team limit is exceeded on any day
                    is_valid = False
                    break
            if is_valid:
                valid_start_dates.append(start_date)
     
        print("Team Availability:", team_availability)
        print("Valid Start Dates:", valid_start_dates)
    
        
        if not valid_start_dates:
            print("No valid start date found. Skipping this engine.")
            continue
           
                    
        start_date = np.random.choice(valid_start_dates)
        end_date = start_date + maintenance_days - 1
    
        print("Selected Start Date:", start_date)
        print("Selected End Date:", end_date)
    
        # Update team availability for the selected team and days
        for day in range(start_date, end_date + 1):
            team_availability[day][team_type] += 1
    
        print("Team Availability Updated:", team_availability)
    
    
        RUL = filtered_df.loc[filtered_df['id'] == engine_id, 'RUL'].values[0]
        safety_due_date = RUL
        print("Safety Due Date:", safety_due_date)
        
    
    
        if end_date > safety_due_date:
            penalty_cost_value = penalty_cost(engine_costs[engine_id - 1], end_date - safety_due_date)
        else:
            penalty_cost_value = 0
    
        print("Penalty Cost Value:", penalty_cost_value)
    
        # Append to individual list
        individual.append({
            'Engine_id': engine_id,
            'Team': team_type, 
            'Safety_due_date': safety_due_date,
            'Start_date': start_date,
            'End_date': end_date,
            'Penalty_cost': penalty_cost_value
        })
    return individual

# Create DataFrame from individual list
schedule_df = pd.DataFrame(generate_random_schedule(T))

print("\nFinal Schedule DataFrame:")
print(schedule_df.sort_values(by='Start_date'))
print("Number of Scheduled Maintenance Tasks:", len(schedule_df))


Engine IDs: [81, 35, 49, 100, 24, 56, 36, 77, 41, 92, 31, 37, 20, 64, 66, 91, 68, 34, 42, 82, 90, 61, 76, 40]

Processing Engine ID: 81
Selected Team Type: A
Maintenance Days Required: 8
Team Availability: {1: {'A': 0, 'B': 0}, 2: {'A': 0, 'B': 0}, 3: {'A': 0, 'B': 0}, 4: {'A': 0, 'B': 0}, 5: {'A': 0, 'B': 0}, 6: {'A': 0, 'B': 0}, 7: {'A': 0, 'B': 0}, 8: {'A': 0, 'B': 0}, 9: {'A': 0, 'B': 0}, 10: {'A': 0, 'B': 0}, 11: {'A': 0, 'B': 0}, 12: {'A': 0, 'B': 0}, 13: {'A': 0, 'B': 0}, 14: {'A': 0, 'B': 0}, 15: {'A': 0, 'B': 0}, 16: {'A': 0, 'B': 0}, 17: {'A': 0, 'B': 0}, 18: {'A': 0, 'B': 0}, 19: {'A': 0, 'B': 0}, 20: {'A': 0, 'B': 0}, 21: {'A': 0, 'B': 0}, 22: {'A': 0, 'B': 0}, 23: {'A': 0, 'B': 0}, 24: {'A': 0, 'B': 0}, 25: {'A': 0, 'B': 0}, 26: {'A': 0, 'B': 0}, 27: {'A': 0, 'B': 0}, 28: {'A': 0, 'B': 0}, 29: {'A': 0, 'B': 0}, 30: {'A': 0, 'B': 0}}
Valid Start Dates: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Selected Start Date: 23
Selected End Da

In [ ]:
import random

from deap import base
from deap import creator
from deap import tools

IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

In [ ]:
ind1 = toolbox.individual()

In [ ]:
print(ind1)               # [0.86..., 0.27..., 0.70..., 0.03..., 0.87...]
print(ind1.fitness.valid) # False